# Interpolation

## Subsection 1b

Continuation of "Interpolate using Arcpy" section in the [1]ActiveFires_California.ipynb 

In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True

In [ ]:
rootPath = "G:\My Drive\California FireTrends (2012-2020)"
env.workspace = rootPath

In [ ]:
import arcgisscripting
gp = arcgisscripting.create()
gp.CheckOutExtension("Spatial")
gp.overwriteoutput = True

os.chdir(rootPath)

In [ ]:
# get list of files based on directory and extension inputs 
def listFiles_ByExt(rootPath, ext):
    file_list = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                file_list.append(os.path.join(path,names))
    return(file_list)

# Create new folder in root path 
def createFolder(rootPath, folderName): 
    folderPath = os.path.join(rootPath, folderName) 
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    return folderPath + "\\" 

In [ ]:
# Connect to google drive directory as G: Drive in local desktop environment with arcpy installed 
merged_byfire = listFiles_ByExt('G:\My Drive\California_FireTrends\Data\FireInstances', '.shp')

In [ ]:
path = 'G:\My Drive\California_FireTrends\Data\Interpolation'
combined_perimeters = 'G:\My Drive\California_FireTrends\Data\Interagency_FirePerimeters\CALIFORNIA\Combined500_2012_2020\Perimeters500_2012_2020.shp'

for i, AF_byFire in enumerate(merged_byfire):
    file_nm = os.path.basename(AF_byFire)
    fire = file_nm.split('_')[0]
    firenm = fire.replace(' ', '')
    year = file_nm.split('_')[1]
    
    final_file_path = os.path.join(path, 'SHP', year, fire, firenm + '_' + year + '_' + 'NAT.shp')
    if os.path.exists(final_file_path) == False: 
        
        try: 
            AF_RASpath = createFolder(path, 'AF_RAS')
            yrpath = createFolder(AF_RASpath, year)
            outpath = createFolder(yrpath, fire)

            print('AF Conversion: ', i, fire, year, os.path.join(outpath, firenm + '_' + year + '_' + 'AFRAS_100m.shp'))
            
            # Represent earliest day of detection for viirs & m6 merged AF layer 
            # Reduces bullseye effect and helps prioritize over earliest hot spot detection 
            AFRAS = arcpy.PointToRaster_conversion(AF_byFire, "JulianDay", 
                                os.path.join(outpath, firenm + '_' + year + '_' + 'AFRAS_100m.tif'), "MINIMUM", "", 375)

            AF_SHPpath = createFolder(path, 'AF_SHP')
            yrpath = createFolder(AF_SHPpath, year)
            outpath = createFolder(yrpath, fire)

            AF_filt = arcpy.RasterToPoint_conversion(AFRAS, 
                                                     os.path.join(outpath, firenm + '_' + year + '_' + 'AFRAS_100m.shp'), 
                                                     'VALUE')

            # Create new folders to save by year and fire name
            raspath = createFolder(path, 'RAS')
            yrpath = createFolder(raspath, year)
            outpath = createFolder(yrpath, fire)

            # Interpolate 
            print('interpolating: ', fire, year)
            outNaturalNeighbor = NaturalNeighbor(AF_filt, "grid_code", 100)
            outInt = Int(outNaturalNeighbor)

            # Mask interpolated raster with interagency perimeter 
            print('clipping: ', fire, year) 

            SQL_clause =  "\"IncidentNa\" = " + "'" + fire + "'" + " AND \"Year\" = {0}".format(int(year)) 
            print(SQL_clause)
            perimeter = arcpy.SelectLayerByAttribute_management(combined_perimeters, "NEW_SELECTION", SQL_clause)

            outExtractByMask = ExtractByMask(outInt, perimeter)
            outMajFilt = MajorityFilter(outExtractByMask, "EIGHT", "MAJORITY")


            outMajFilt.save(os.path.join(outpath, firenm + '_' + year + '_' + 'NAT.tif'))

            # Create new folders to save by year and fire name
            shppath = createFolder(path, 'SHP')
            yrpath = createFolder(shppath, year)
            outpath = createFolder(yrpath, fire)

            print('Convert to Poly: ', fire, year)
            # Convert interpolated surface to polygon layer
            shp_out = arcpy.RasterToPolygon_conversion(outExtractByMask, 
                                             os.path.join(outpath, firenm + '_' + year + '_' + 'NAT.shp') , 
                                             "NO_SIMPLIFY", "VALUE")

            # Smooth out edges of polygon
            arcpy.SmoothSharedEdges_cartography(shp_out, "PAEK", "5 Kilometers")

            print('saved ', firenm + '_' + year + '_' + 'NAT.shp')
        except:
                print('error: ', fire, year)
                continue
            